In [24]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import os
import cv2
import numpy as np
import pandas as pd
import sys  
df_zone = pd.read_csv('file1.csv')

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

def download_images():
#     base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/test_images/'
#     filenames = ['image1.jpg', 'image2.jpg']
    base_url = 'workspace/training_demo/images/train/'
    
    filenames = []
    for root, directories, file in os.walk(base_url):
        for i in range(120, 300):
            if(file[i].endswith(".jpg")):
                print(os.path.join(file[i]))
                filenames.append(os.path.join(file[i]))
#     filenames = ['evraz 120.jpg', 'evraz 250.jpg']
    image_paths = []
    for filename in filenames:
#         image_path = tf.keras.utils.get_file(fname=filename,
#                                             origin=base_url + filename,
#                                             untar=False)
        image_path = base_url + filename
        image_path = pathlib.Path(image_path)
        image_paths.append(str(image_path))
    return image_paths

IMAGE_PATHS = download_images()

In [26]:
# # Download and extract model
# def download_model(model_name, model_date):
#     base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
#     model_file = model_name + '.tar.gz'
#     model_dir = tf.keras.utils.get_file(fname=model_name,
#                                         origin=base_url + model_date + '/' + model_file,
#                                         untar=True)
#     return str(model_dir)

# MODEL_DATE = '20200711'
# MODEL_NAME = 'covsh_net-32'
# PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
PATH_TO_MODEL_DIR = r'workspace/training_demo/exported-models/my_model/'
PATH_TO_PEOPLE_MODEL = r'workspace\training_demo\pre-trained-models\ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model'

In [28]:
# # Download labels file
# def download_labels(filename):
#     base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
#     label_dir = tf.keras.utils.get_file(fname=filename,
#                                         origin=base_url + filename,
#                                         untar=False)
#     label_dir = pathlib.Path(label_dir)
#     return str(label_dir)

PATH_TO_LABELS = 'workspace/training_demo/annotations/'

In [30]:
LABEL_FILENAME = 'label_map.pbtxt'

In [32]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

detect_people = tf.saved_model.load(PATH_TO_PEOPLE_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


Loading model...Done! Took 35.278002977371216 seconds


In [33]:
category_index = {1:{'id': 1, 'name': 'Dangerous0'}, 2:{'id': 2, 'name': 'Dangerous1'}, 3:{'id': 3, 'name': 'Fastening0'}, 4:{'id': 4, 'name': 'Fastening1'}}

In [34]:
category_index_people = {1:{'id': 1, 'name': '/m/01g317', 'display_name': 'people'}}

In [35]:
PATH_TO_VIDEO = "C:/Tensorflow/video/2021-12-03 07-07-48.avi"

In [36]:
def findDistance(line, x_center_line, y_center_line):
    x1, y1, x2, y2 = line
    under_line = x2 - x1 if x2 > x1 else x1 - x2
    small_under_line = x2 - x_center_line if y2 > y1 else x_center_line - x1
    vertial_line = y1 - y2 if y1 > y2 else y2 - y1
    y_lowest = y1 if y1 < y2 else y2
    height = under_line * vertial_line / small_under_line
    y_peresek_line = y_lowest + height
    return y_peresek_line - y_center_line


def findNearestLine(df, x_center_line, y_center_line):
    nearest_index = 0
    min_distance = sys.maxsize
    for i in range(len(df)):
        lh_x, lh_y, rh_x, rh_y, ld_x, ld_y = df.iloc[i, 1], df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4], df.iloc[i, 5], df.iloc[i, 6]
        line1 = [lh_x, lh_y, rh_x, rh_y]
        line2 = [ld_x, ld_y, lh_x, lh_y]
        lines = [line1, line2]
        for line in lines:
            if line[0] < x_center_line < line[2]:
                cur_dist = findDistance(line, x_center_line, y_center_line)
                if cur_dist < min_distance:
                    nearest_index = i
                    min_distance = cur_dist

    return nearest_index


def zoneDrawer(df, img, i):
#     for i in range(len(df)):
    mas = np.array([[df['lh_x'][i], df['lh_y'][i]], [df['rh_x'][i], df['rh_y'][i]], [df['rd_x'][i], df['rd_y'][i]], [df['ld_x'][i], df['ld_y'][i]]], np.int32)
    mas = mas.reshape((-1, 1, 2))
    image = cv2.polylines(img, [mas], True, (0, 0, 255), 4)
    return image


# img = cv2.imread(r'evraz 432.jpg')
x_y = []
def on_EVENT_LBUTTONDOWN(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        x_y.append(x)
        x_y.append(y)
        xy = "%d,%d" % (x, y)
        cv2.circle(img, (x, y), 1, (255, 0, 0), thickness = -1)
        cv2.putText(img, xy, (x, y), cv2.FONT_HERSHEY_PLAIN,
                    1.0, (0,0,0), thickness = 1)
        cv2.imshow("image", img)
        print(x_y)
# cv2.namedWindow("image")
# cv2.setMouseCallback("image", on_EVENT_LBUTTONDOWN)

# # i = 0


# while (1):
#     cv2.imshow('image', image)
#     if cv2.waitKey(20) & 0xFF == 27:
#         print(findNearestLine(df, 1186, 425))
#         break
# cv2.destroyAllWindows()

In [37]:
def isLineInOtherLine(line, x_center_line, y_up_center_line, man_length):
    x1, y1, x2, y2 = line
    under_line = x2 - x1 if x2 > x1 else x1 - x2
    small_under_line = x2 - x_center_line if y2 > y1 else x_center_line - x1
    vertial_line = y1 - y2 if y1 > y2 else y2 - y1
    y_lowest = y1 if y1 < y2 else y2
    height = under_line * vertial_line / small_under_line
    y_peresek_line = y_lowest + height - y_up_center_line
    print(y_peresek_line, man_length)
    return True if man_length > y_peresek_line else False

In [38]:
def isManPeresekLine(df, x_center_line, y_up_center_line, i, man_length):
    nearest_index = 0
    lh_x, lh_y, rh_x, rh_y, ld_x, ld_y = df.iloc[i, 1], df.iloc[i, 2], df.iloc[i, 3], df.iloc[i, 4], df.iloc[i, 5], df.iloc[i, 6]
    line1 = [lh_x, lh_y, rh_x, rh_y]
    line2 = [ld_x, ld_y, lh_x, lh_y]
    lines = [line1, line2]
    for line in lines:
        if line[0] < x_center_line < line[2]:
            peresek = isLineInOtherLine(line, x_center_line, y_up_center_line, man_length)
            if peresek == True:
                return True
    return False

In [41]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


video = cv2.VideoCapture(PATH_TO_VIDEO)
while(video.isOpened()):
    
    ret, image_np = video.read()
    
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    
    image_tensor = np.expand_dims(image_np, axis=0)
    
    
    input_tensor = tf.convert_to_tensor(image_tensor)
    
    #------------------
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    #------------------
    people = detect_people(input_tensor)
    
    num_detections = int(people.pop('num_detections'))
    
    people = {key: value[0, :num_detections].numpy()
                   for key, value in people.items()}
    people['num_detections'] = num_detections
    
    people['detection_classes'] = people['detection_classes'].astype(np.int64)

    
#     people_classes = np.array([])
#     people_boxes = np.array([])
#     people_scores = np.array([])
    
#     for i in range(0, len(people['detection_classes'])):
#         if(people['detection_classes'][i] == 1):
#             np.delete(people['detection_classes'], people['detection_classes'][i])
#             np.delete(people['detection_boxes'], people['detection_boxes'][i])
#             np.delete(people['detection_scores'], people['detection_scores'][i])
    
    #------------------
    
    image_np_with_detections = image_np.astype('uint8')
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'],
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=4,
            min_score_thresh=.40,
            agnostic_mode=False)
    
#     viz_utils.visualize_boxes_and_labels_on_image_array(
#             image_np_with_detections,
#             people_boxes,
#             people_classes,
#             people_scores,
#             category_index_people,
#             use_normalized_coordinates=True,
#             max_boxes_to_draw=4,
#             min_score_thresh=.40,
#             agnostic_mode=False)
#     #---------------------------------
    boxes = detections['detection_boxes']
    max_boxes_to_draw = 4
    scores = detections['detection_scores']
    min_score_thresh=.40
    
    x_center = 0
    y_center = 0
    
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores is None or scores[i] > min_score_thresh:
            ymax, xmin, ymin, xmax = boxes[i]
            print(xmin, xmax, ymin, ymax)
            im_height, im_width, _ = image_np_with_detections.shape

            (x1, x2, y1, y2) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)

            x_center = (x2 + x1) / 2
            y_center = (y2 + y1) / 2
            
#             image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), thickness = 3)
    
    boxes = people['detection_boxes']
    scores = people['detection_scores']
    
    x_center = 0
    y_center = 0
    
    dangerous_zone = findNearestLine(df_zone, x_center, y_center)
    image_np_with_detections = zoneDrawer(df_zone, image_np_with_detections, dangerous_zone)
    
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores is None or scores[i] > min_score_thresh:
            if people['detection_classes'][i] == 1:
                ymax, xmin, ymin, xmax = boxes[i]
                print(xmin, xmax, ymin, ymax)
                im_height, im_width, _ = image_np_with_detections.shape

                (x1, x2, y1, y2) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
                
                p_center = ((x1 + x2) / 2, y2)
                
                zone = df_zone.iloc[dangerous_zone]
                man_length = y2 - y1 if y2 > y1 else y1 - y2
                print('igreki', y1, y2, x1, x2)
                if(isManPeresekLine(df_zone, x1, y2, dangerous_zone, man_length)):
                    image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), thickness = 3)
                else:
                    image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), thickness = 3)
                
#                 if((p_center[1] < zone[2]) and (p_center[1] < zone[5]) and (p_center[1] > zone[6]) and (p_center[1] > zone[8])):
#                     if((p_center[0] > zone[1]) and (p_center[0] < zone[3]) and (p_center[0] > zone[5]) and (p_center[0] < zone[7])):    
#                         image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), thickness = 3)
#                     else:    
#                         image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), thickness = 3)
#                 else:
#                     image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), thickness = 3)
            
              
#             image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), thickness = 3)
#             cv2.circle(image_np_with_detections, (int(x_center), int(y_center)), 1, (255, 0, 0), thickness = 3)
    
#             class_name = category_index[output_dict['detection_classes'][i]]['name']
#             print ("This box is gonna get used", boxes[i], output_dict['detection_classes'][i])
    #---------------------------------
    
#     index_max_score = list(detections['detection_scores']).index(max(detections['detection_scores']))
#     xmin, xmax, ymin, ymax = detections['detection_boxes'][index_max_score]
#     im_height, im_width, _ = image_np_with_detections.shape
    
#     (x1, x2, y1, y2) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
    
#     x_center = (x2 + x1) / 2
#     y_center = (y2 + y1) / 2
    
#     image_np_with_detections = cv2.rectangle(image_np_with_detections, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), thickness = 3)
#     cv2.circle(image_np_with_detections, (int(x_center), int(y_center)), 1, (255, 0, 0), thickness = 3)
    
    cv2.imshow('Object detector', image_np_with_detections)

    # Press 'q' to quit
    if cv2.waitKey(1) == ord('q'):
        break

# Clean up
video.release()
cv2.destroyAllWindows()